# Bring your own components (BYOC)

Starting in V4 Clara train is based of MONAI 
from their website 
"The MONAI framework is the open-source foundation being created by Project MONAI. 
MONAI is a freely available, community-supported, 
PyTorch-based framework for deep learning in healthcare imaging. 
It provides domain-optimized foundational capabilities for developing healthcare imaging training workflows in a native PyTorch paradigm."
<br><img src="screenShots/MONAI.png" alt="Drawing" style="height: 200px;width: 400px"/><br>


Clara Train SDK is modular and flexible enough to allow researchers to bring their own components including:
1. [Transformations](https://docs.monai.io/en/latest/transforms.html#) 
2. [Loss functions](https://docs.monai.io/en/latest/losses.html)
3. [Model Architecture](https://docs.monai.io/en/latest/networks.html)
4. [Loaders](https://docs.nvidia.com/clara/tlt-mi/clara-train-sdk-v3.0/nvmidl/byom.html#bring-your-own-data-loader)
5. [Metrics](https://docs.monai.io/en/latest/metrics.html)

By the end of this notebook you should be able to bring your own components mentioned above.


## Prerequisites
- Familiar with Clara train main concepts. See [Getting Started Notebook](../GettingStarted/GettingStarted.ipynb)
- Nvidia GPU with 8Gb of memory   


## Dataset 
This notebook uses a sample dataset (ie. a single image of a spleen dataset) provided in the package to train a small neural network for a few epochs. 
This single file is duplicated 32 times for the training set and 9 times for the validation set to mimic the full spleen dataset. 
    

# Lets get started
It is helpful to check that we have an NVIDIA GPU available in the docker by running the cell below

In [ ]:
# following command should show all gpus available 
!nvidia-smi

## 1.1 General Concept
You can easily BYOC into Clara Train SDK by writing your python code then point to it in the `config.json` using `path` instead of the `name` tag. 
Throughout this notebook we have placed all of our examples from our documentations into the [BYOC](BYOC) folder. 

Normal | BYOC  
 --- | ---  
{<br>"name": "CropFixedSizeRandomCenter", <br> "args": {"fields": "image"}<br> } | { <br> "path": "myTransformation.MyAddRandomConstant", <br> "args": {"fields": "image"}<br> } 

 
We modified the [set_env.sh](commands/set_env.sh) to include the path. 
Let us run the cells below that define some helper functions we will be using and see where we added the BYOC to the pythonpath


In [ ]:
MMAR_ROOT="/claraDevDay/GettingStarted/"
print ("setting MMAR_ROOT=",MMAR_ROOT)
%ls $MMAR_ROOT

!chmod 777 $MMAR_ROOT/commands/*
def printFile(filePath,lnSt,lnEnd):
    print ("showing ",str(lnEnd-lnSt)," lines from file ",filePath, "starting at line",str(lnSt))
    !< $filePath head -n "$lnEnd" | tail -n +"$lnSt"

## 1.2 Add BYOC folder to PYTHONPATH 
It is important to add the folder containing your code to the PYTHONPATH variable.
The easiest way to do this is to add it to the `set_env.sh` file since it is called from all the train commands.
Let's take a look at this `set_env.sh` file 

In [ ]:
printFile(MMAR_ROOT+"/commands/set_env.sh",0,20)

## 2.1 BYO Transformation: Adding random noise to image pixels
Now lets write a full transformation `MyRandAdditiveNoised` from scratch. For this you need to:
1. Implement `Randomizable` and `MapTransform`
2. Define `__call__` function. 
Also define `set_random_state` and `randomize` functions for Randomizable

see how we did this in by running the cell below

In [ ]:
printFile(MMAR_ROOT+"/config/myTransformation.py",16,30)


Now to run this we need to modify the train config by setting the `path` 
to our newly created transformation `myTransformation.MyRandAdditiveNoised`. 
We also would like to debug the output so we will add the `SaveImageD` Transform. 
This transform would pause the training and save batches to `output_dir` for us to check.  

In [ ]:
configFile=MMAR_ROOT+"/custom/trn_BYOC_transform.json"
printFile(configFile,0,50)


## 2.2 Run and see Debugging Data
So let us now run training and see the results 

In [ ]:
! $MMAR_ROOT/commands/train_W_Config.sh trn_BYOC_transform.json


Now let us see the sample images in the debug folder  

In [ ]:
! ls -la /claraDevDay/Data/_tmpDebugPatches/
! ls -la /claraDevDay/Data/_tmpDebugPatches/spleen_8


## 3. BYO Network Architecture and Loss
Clara Train SDK also allows you to write your own network architecture as well as your loss function. 
In this example we have a shallow Unet architecture defined in [myNetworkArch.py](BYOC/myNetworkArch.py) 
as well as our own dice loss defined in [myLoss.py](BYOC/myLoss.py). 

Normal | BYOC  
 --- | --- 
"loss": {<br> "name": "DiceLoss",<br> "args":{ ...      } <br>}, | "loss": {<br> **"path"**: "myLoss.MyDiceLoss",<br>  "args": {... }<br>} |
"model": {<br> "name": "UNet",<br>"args": { ... }<br>}, | "model": {<br>**"path"**: "myNetworkArch.MyBasicUNet",<br>"args": { ... }<br>},
 

Let us see how it is defined   

In [ ]:
printFile(MMAR_ROOT+"/custom/myNetworkArch.py",0,30)
printFile(MMAR_ROOT+"/custom/myLoss.py",0,30)

Let us Examine the config file [trn_BYOC_Arch_loss.json](config/trn_BYOC_Arch_loss.json) 

In [ ]:
configFile=MMAR_ROOT+"/config/trn_BYOC_Arch_loss.json"
printFile(configFile,11,18)
printFile(configFile,32,43)


Now let us train our network  

In [ ]:

! $MMAR_ROOT/commands/train_W_Config.sh trn_BYOC_Arch_loss.json


# 4. Exercise 

### 4.1. BYO Data Loader
For this example we will see how to use a custom loader specifically to load a numpy file. 
To do this, we first load our nii.gz file and save it a np.  

In [ ]:
import nibabel as nib
import numpy as np
dataRoot="/claraDevDay/Data/sampleData/"
for imgLb in ["imagesTr","labelsTr"]:
    filename= dataRoot+imgLb+"/spleen_8.nii.gz"
    img = nib.load(filename)
    data = img.get_fdata()
    np.save(filename[:-7]+".npy",data)
!ls -ls $dataRoot/imagesTr
!ls -ls $dataRoot/labelsTr

Now you should:
1. Modify the environment file to point to [datasetNp.json](../Data/sampleData/datasetNp.json)
2. write a numpy dataloader similar to the one in monai [NumpyReader](https://docs.monai.io/en/latest/_modules/monai/data/image_reader.html#NumpyReader)
3. Change dataloader transformation to point.

### 4.2. Modify custom loss

Modify the custom loss file to be a weighted dice loss per label.

Some Tips:

1. You can add code below [myLoss.py](custom/myLoss.py) in the init function
```
    # uncomment lines below to enable label weights
     self.label_weights=label_weights
     if self.label_weights is not None:
         self.label_weights=[x / sum(self.label_weights) for x in self.label_weights]
         print ("====== AEH applying label weights {} refactored as {}".format(label_weights,self.label_weights))
```
2. Similarly uncomment the lines in the `forward` function to multiply the weights given with the loss  
``` 
 if self.label_weights is not None:  # add wights to labels
     bs=intersection.shape[0]
     w = torch.tensor(self.label_weights, dtype=torch.float32,device=torch.device('cuda:0'))
     w= w.repeat(bs, 1) ## change size to [BS, Num of classes ]
     intersection = w* intersection
```
3. You need to pass the weights by adding `label_weights` in the args of your loss in the training config

